# Fitting an early MLE
In this notebook we will fit an early MLE model to audiovisual speech perception data.

In [11]:
import numpy as np
from os import path
from glob import glob

import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

from scipy.stats import norm
from scipy.optimize import minimize, LinearConstraint, Bounds

## Prepare Data
The data consists of five text files, each containing seven rows and five columns, where:
- Row 1: Audiotorial data
- Row 2: Visual data
- Rows 3-7: Audiovisual data
    - a combination of rows 1 and 2
    - visual goes from 'b' (row 3) to 'd' (row 7) 
    - audio goes from 'b' (col 1) to 'd' (col 5)

In [2]:
# get paths to data files
file_paths = glob("./data/*.txt")

# load all data into a single array
data = np.array([np.loadtxt(fname) for fname in file_paths])
N, M, K = data.shape

# define number of samples for each subject
n_samples = 24 

# split into the three different data types
data_A = data[:, 0, :]
data_V = data[:, 1, :]
data_AV = data[:, 2:, :]

## Fit MLE

In [28]:
def compute_params(c_A, c_V, std_A, std_V):
    """Compute the parameters for the MLE model from the free parameters"""

    # get variances from standard deviations
    var_A = std_A ** 2
    var_V = std_V ** 2
    
    # compute denominator for calculations
    den = 1 / (var_A + var_V) 

    # compute mean and variance
    mu_AV = np.zeros((5, 5))
    for i in np.arange(5):
        for j in np.arange(5):
            mu_AV[i, j] = ((i+1 - c_A) * var_V + (j+1 - c_V) * var_A) * den
    var_AV = var_A * var_V * den

    # return mean and standard deviation
    return mu_AV, np.sqrt(var_AV)

In [66]:
def objective_function(theta, data):
    """Compute MLE objective function on data (7, 5) from a single subject
        where theta=[c_A, c_V, std_A, std_V].
    """
    # extract parameters
    c_A, c_V, std_A, std_V = theta

    # compute parameters for audiovisual
    mu_AV, std_AV = compute_params(c_A, c_V, std_A, std_V)

    # compute the probabilities
    p_A = norm.pdf(data[0], np.arange(5) + 1, std_A)
    p_V = norm.pdf(data[1], np.arange(5) + 1, std_V)
    p_AV = norm.pdf(data[2:], mu_AV, std_AV)

    # compute the log-likelihoods

    # L = L_A + L_V + L_AV

    # return the negative log-likelihood
    return -L

In [67]:
def mle_fit(data):
    """Perform MLE fit to data (7, 5) for a single subject"""
    theta = np.random.uniform(low=1, high=10, size=4)
    opt_result = minimize(objective_function, theta, args=(data))

    objective = opt_result.fun
    c_A, c_V, std_A, std_V = opt_result.x

    return objective, c_A, c_V, std_A, std_V

In [68]:
mle_fit(data[0])

(126.69748843548382,
 -15.430128402568867,
 -4.388938836581458,
 9.568175724416966,
 15.010315618578769)